In [1]:
import os, sys
import subprocess as sp

# add custom modules and astro data path 
pp = '../nav_tools/'
sys.path.append(pp)

# needed by basemap
os.environ['PROJ_LIB'] = '/opt/miniconda3/envs/astrolab/share/proj'

In [2]:
import numpy as np
import matplotlib.pyplot as mpl
mpl.rcParams['text.usetex'] = True
mpl.rcParams.update({'font.size': 7})

import celestialdata as cdata
import navigationalstars as ns
import navtools as nt

In [3]:
def findVisibleStars(date, time, pos, h0, h1, ns_db):
    
    t = [date[2],date[1],date[0],time[0],time[1],time[2]]
    star_ids = ns_db.keys()
    
    vs = []
    for s_id in star_ids:
        sdd = cd.get_star_data(s_id,t)
        saz = cd.get_star_altaz(s_id,t,pos)
        if saz['alt'] > h0 and saz['alt'] < h1:
            s_name = ns_db[s_id][0]
            vs.append({'name':s_name, 'alt':saz['alt'], 'az':saz['az'], 'dec':sdd['dec'], 'sha':sdd['sha'], 'sha':sdd['sha']})
    
    return vs

In [4]:
def generateLatexInput(date,time,pos,stars,gha_a,obs_h,h0,h1):
    
    fo = open('body.tex','w')
    
    str_fi = r"$\varphi={:02d}^\circ \, {:04.1f}' \, \mathrm{{{:s}}}$".format(pos[0][0],pos[0][1],pos[0][2])
    str_la = r"$\lambda={:03d}^\circ \, {:04.1f}' \, \mathrm{{{:s}}}$".format(pos[1][0],pos[1][1],pos[1][2])
    
    fo.write(r'\textbf{{Position}}: {:s}, {:s}\\[2mm]'.format(str_fi, str_la)); fo.write('\n')
    fo.write(r'\textbf{{Date}} ({{\footnotesize dd/mm/yyyy}}): {:02d}/{:02d}/{:04d}\\[2mm]'.format(date[0],date[1],date[2])); fo.write('\n')
    fo.write(r'\textbf{{Time}}: {:02d}:{:02d}:{:02d}\\[2mm]'.format(time[0],time[1],time[2])); fo.write('\n')
    
    gha_a_dm = nt.dd2dms(gha_a)
    str_gha_a = r"${:02d}^\circ \, {:04.1f}'$".format(int(gha_a_dm[0]), gha_a_dm[1])
    fo.write(r'$\textbf{{\textsc{{Gha}}}}_{{\aries}}:$ {:s}\\[2mm]'.format(str_gha_a)); fo.write('\n')
    fo.write(r'\textbf{{Observer height}}: {:04.1f}$\,$m\\[2mm]'.format(obs_h)); fo.write('\n')
    fo.write(r'\textbf{{Altitude limit}}: $h_{{\mathrm{{min}}}}={:02d}^\circ$; $h_{{\mathrm{{\max}}}}={:02d}^\circ$\\'.format(int(h0),int(h1))); fo.write('\n')
    
    fo.write(r'\begin{center}'); fo.write('\n')
    fo.write(r'   \includegraphics[width=16cm]{visible_stars.pdf}'); fo.write('\n')
    fo.write(r'\end{center}'); fo.write('\n')
    
    fo.write(r'\vspace{5mm}'); fo.write('\n')
    fo.write(r'\begin{center}'); fo.write('\n')
    fo.write(r'\footnotesize'); fo.write('\n')
    fo.write(r'\begin{tabular}{l l r r r r}\hline'); fo.write('\n')
    fo.write(r' \multicolumn{1}{c}{\#} & \multicolumn{1}{c}{Name} & \multicolumn{1}{c}{$h$} & \multicolumn{1}{c}{$\omega$} & \multicolumn{1}{c}{$\delta$} & \multicolumn{1}{c}{\textsc{Sha}}\\ \hline'); fo.write('\n')
    i = 1
    for star in stars:
        h_dm = nt.dd2dms(star['alt'])
        str_h = r"${:02d}^\circ \, {:04.1f}'$".format(int(h_dm[0]), h_dm[1])
        str_w = r'${:05.1f}^\circ$'.format(star['az'])
        dec_dm = nt.dd2dms(star['dec'])
        if star['dec'] < 0:
            str_dec = r"${:02d}^\circ \, {:04.1f}' \, \mathrm{{S}}$".format(int(np.abs(dec_dm[0])), dec_dm[1])
        else:
            str_dec = r"${:02d}^\circ \, {:04.1f}' \, \mathrm{{N}}$".format(int(np.abs(dec_dm[0])), dec_dm[1])
        sha_dm = nt.dd2dms(star['sha'])
        str_sha = r"${:02d}^\circ \, {:04.1f}'$".format(int(sha_dm[0]), sha_dm[1])
        fo.write(r'   {:02d} & {:s} & {:s} & {:s} & {:s} & {:s}\\'.format(i, star['name'], str_h, str_w, str_dec, str_sha)); fo.write('\n')
        
        i += 1
    fo.write(r'\hline'); fo.write('\n')
    fo.write(r'\end{tabular}'); fo.write('\n')
    fo.write(r'\end{center}'); fo.write('\n')
    
    fo.close()
    
    # run latex compile and clean
    p = sp.Popen(['xelatex','today_stars.tex'], stdout = sp.PIPE)
    p.wait()

In [5]:
cd = cdata.CelestialData(pp)
ns_db = cd.get_nav_stars_db()

date = [9,11,2022]
time = [20,0,0]
fi = [46,6.08,'N']
la = [13,38.85,'E']
h = 20.0
pos = [nt.nav2dd(fi), nt.nav2dd(la), h]

h0 = 20.0
h1 = 80.0
stars_vis = findVisibleStars(date, time, pos, h0, h1, ns_db)
stars_vis.sort(key=lambda k : k['az'])

i = 1
data = []
for star in stars_vis:
    print('{:02d} - {:11s}  h = {:s}  {:s} = {:s}'.format(i,star['name'],nt.prettyPrintAlt(star['alt']), nt.u_omega, nt.prettyPrintAz(star['az'])))
    data.append([star['az'],star['alt']])
    i += 1
    
data = np.array(data)

fig, ax = mpl.subplots()
i = 1
for da in data:
    ax.plot(da[0], da[1], marker='*', markersize=3, color='red')
    ax.text(da[0]-4, da[1]+1, '{:02d}'.format(i))
    i += 1
ax.grid()
ax.set_xlabel('azimuth - $\omega$[deg]')
ax.set_ylabel('altitude - $h$[deg]')
ax.set_xlim([-10, 410])
ax.set_ylim([h0-5, h1+5])
fig.tight_layout()
fig.savefig('visible_stars.pdf')

OSError: cannot download ftp://ftp.iers.org/products/eop/rapid/standard/finals2000A.all because <urlopen error ftp error: EOFError()>

In [ ]:
# generate latex document
t = [date[2],date[1],date[0],time[0],time[1],time[2]]
gha_a = cd.get_aries_gha(t)

generateLatexInput(date,time,[fi,la],stars_vis,gha_a,h,h0,h1)
os.system("rm -rf body.tex visible_stars.pdf *.aux *.out *.log")